## Mounting with Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/




---



---
## Importing Libraries


In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from zipfile import ZipFile
import pickle
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint



---



---

## Looking over to the DATA

In [ ]:
def unzip():
  zip = ZipFile('/content/drive/MyDrive/Colab Notebooks/Pneumonia Detection/archive.zip', 'r')
  zip.extractall('Unzip_File')
  zip.close()

# unzip()

In [ ]:
# # path name
train_dir = '/content/Unzip_File/chest_xray/train'
val_dir = '/content/Unzip_File/chest_xray/val'
test_dir = '/content/Unzip_File/chest_xray/test'

# categories
categories = ['NORMAL', 'PNEUMONIA']

In [ ]:
image_width = 224
image_height = 224
image_size = (image_width,image_height)

def make_data():
  nor_data=[]
  pne_data=[]
  
  for dir in [train_dir, val_dir, test_dir]:
    for category in categories:
      path = os.path.join(dir, category)
      label = categories.index(category)
      
      for img in os.listdir(path):
        img_path = os.path.join(path,img)
          
        try:
          image = cv2.imread(img_path)
          image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
          image = cv2.resize(image, image_size)

          image = np.array(image, dtype=np.float64)
          if label==0:
            nor_data.append([image,label])
          else:
            pne_data.append([image,label])
          
        except:
          pass
  pik = open('/content/drive/MyDrive/Colab Notebooks/Chest XRay/normal.pickle', 'wb')
  pickle.dump(nor_data, pik)
  pik.close()
  pik = open('/content/drive/MyDrive/Colab Notebooks/Chest XRay/pneumonia.pickle', 'wb')
  pickle.dump(pne_data, pik)
  pik.close()
         
# make_data()       

In [ ]:
nor_count=0
pne_count = 0
def load_data(dir):
  nor_count=0
  pne_count = 0
  pick = open(dir, 'rb')
  data = pickle.load(pick)
  pick.close()

  np.random.shuffle(data)
  
  feature = []
  labels = []

  for img,label in data:
    feature.append(img)
    labels.append(label)
    if(label==0):
      nor_count+=1
    else:
      pne_count+=1

  feature = np.array(feature, dtype=np.float64)
  labels = np.array(labels)

  feature /= 255.0

  return [feature, labels]

In [ ]:
normal = load_data('/content/drive/MyDrive/Colab Notebooks/Chest XRay/normal.pickle')
pneumonia = load_data('/content/drive/MyDrive/Colab Notebooks/Chest XRay/pneumonia.pickle')

In [ ]:
def split_data():
  train_nor = normal[0][:int(len(normal[0])*0.6)]                                         # 60% normal data
  train_nor_label = normal[1][:int(len(normal[1])*0.6)]
  val_nor = normal[0][int(len(normal[0])*0.6):int(len(normal[0])*0.75)]                    # 15% normal data
  val_nor_label = normal[1][int(len(normal[1])*0.6):int(len(normal[1])*0.75)]
  test_nor = normal[0][int(len(normal[0])*0.75):]                                          # 25% normal data
  test_nor_label = normal[1][int(len(normal[1])*0.75):]

  train_pneu = pneumonia[0][:int(len(pneumonia[0])*0.6)]                                  # 60% pneumonia data
  train_pneu_label = pneumonia[1][:int(len(pneumonia[1])*0.6)]
  val_pneu = pneumonia[0][int(len(pneumonia[0])*0.6):int(len(pneumonia[0])*0.75)]         # 15% pneumonia data
  val_pneu_label = pneumonia[1][int(len(pneumonia[1])*0.6):int(len(pneumonia[1])*0.75)]
  test_pneu = pneumonia[0][int(len(pneumonia[0])*0.75):]                                  # 25% pneumonia data
  test_pneu_label = pneumonia[1][int(len(pneumonia[1])*0.75):]

  train_feature = np.concatenate((train_nor, train_pneu))
  train_label = np.concatenate((train_nor_label, train_pneu_label))

  val_feature = np.concatenate((val_nor, val_pneu))
  val_label = np.concatenate((val_nor_label, val_pneu_label))

  test_feature = np.concatenate((test_nor, test_pneu))
  test_label = np.concatenate((test_nor_label, test_pneu_label))

  return train_feature, train_label, val_feature, val_label, test_feature, test_label

In [ ]:
x_train, y_train, x_val, y_val, x_test, y_test = split_data()

In [ ]:
datagen = ImageDataGenerator(
        # rotation_range = 45,
        # shear_range = 0.2,
        rescale=1/255.0,
        zoom_range = 0.2,
        height_shift_range = 0.2,
        width_shift_range = 0.2,
        fill_mode = "constant")
        # horizontal_flip = True,
        # vertical_flip = True)
        # brightness_range = (0.5, 1.5))

In [ ]:
it = datagen.flow(x_train.reshape(x_train.shape[0], image_height, image_width, 1), y_train, batch_size=64)

In [ ]:
@title
input_layer = tf.keras.Input([image_width, image_height,1])

conv1 = tf.keras.layers.Conv2D(filters=32, kernel_size=(5,5),padding='Same', 
                                 activation='relu')(input_layer)

pool1 = tf.keras.layers.MaxPooling2D(pool_size=(2,2))(conv1)

# drop1 = tf.keras.layers.Dropout(0.25)(conv1)

conv2 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), padding='Same',
                               activation='relu')(pool1)
            
# drop2 = tf.keras.layers.Dropout(0.25)(conv2)

pool2 = tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2))(conv2)

conv3 = tf.keras.layers.Conv2D(filters=96, kernel_size=(3,3), padding='Same',
                               activation='relu')(pool2)

pool3 = tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2))(conv3)

flt1 =tf.keras.layers.Flatten()(pool3)

dn1 = tf.keras.layers.Dense(128, activation='relu')(flt1)

out = tf.keras.layers.Dense(1, activation='sigmoid')(dn1)

In [ ]:
#@title
model = tf.keras.Model(input_layer, out)

In [ ]:
#@title
from keras.applications.vgg19 import VGG19
base_model = VGG19(input_shape = (image_height, image_width, 1), include_top=False, weights=None)
for layer in base_model.layers:
  layer.trainable = False

In [ ]:
#@title
x = tf.keras.layers.Flatten()(base_model.output)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
predictions = tf.keras.layers.Dense(1, activation='sigmoid')(x)

In [ ]:
#@title
model = tf.keras.Model(inputs = base_model.input, outputs = predictions)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=opt, loss='binary_crossentropy',       
              metrics=['accuracy'])

In [ ]:
earlystopping = EarlyStopping(
    monitor = 'val_accuracy',
    patience = 5,
    restore_best_weights=False)

# mcp_save = ModelCheckpoint(
#     '/content/mdl_wts.hdf5', 
#     save_best_only=True, 
#     monitor='loss', 
#     mode='min')

rlrop = ReduceLROnPlateau(
    monitor='val_accuracy', 
    factor=0.2, 
    patience=2)

In [ ]:
model.fit(#it,
          x_train,y_train,
          # steps_per_epoch=len(x_train) // 64,
          validation_data=(x_val,y_val),
          epochs=100,
          callbacks=[earlystopping, rlrop])

Epoch 1/100
110/110 [==============================] - 46s 407ms/step - loss: 0.5837 - accuracy: 0.7235 - val_loss: 0.5274 - val_accuracy: 0.7292 - lr: 0.0010
Epoch 2/100
110/110 [==============================] - 44s 403ms/step - loss: 0.3666 - accuracy: 0.8220 - val_loss: 0.2335 - val_accuracy: 0.9101 - lr: 0.0010
Epoch 3/100
110/110 [==============================] - 44s 403ms/step - loss: 0.2402 - accuracy: 0.9026 - val_loss: 0.2164 - val_accuracy: 0.9033 - lr: 0.0010
Epoch 4/100
110/110 [==============================] - 44s 403ms/step - loss: 0.2031 - accuracy: 0.9243 - val_loss: 0.3011 - val_accuracy: 0.8737 - lr: 0.0010
Epoch 5/100
110/110 [==============================] - 44s 403ms/step - loss: 0.1725 - accuracy: 0.9396 - val_loss: 0.1557 - val_accuracy: 0.9397 - lr: 2.0000e-04
Epoch 6/100
110/110 [==============================] - 44s 403ms/step - loss: 0.1635 - accuracy: 0.9411 - val_loss: 0.1596 - val_accuracy: 0.9317 - lr: 2.0000e-04
Epoch 7/100
110/110 [=================

In [ ]:
model.evaluate(x_test, y_test,verbose=1)

46/46 [==============================] - 18s 392ms/step - loss: 0.1460 - accuracy: 0.9413


[0.14600317180156708, 0.9412969350814819]

In [ ]:
model.save('/content/drive/MyDrive/Colab Notebooks/Pneumonia Detection/model_alpha1.h5')